In [1]:
import s3fs
import os

In [2]:
fs = s3fs.S3FileSystem(
            client_kwargs={
                "endpoint_url": f"""https://{os.environ["AWS_S3_ENDPOINT"]}"""
            }
        ) 

In [3]:
from src.config import (
    CHROMA_DB_LOCAL_DIRECTORY,
    S3_BUCKET,
)

In [4]:
s3_bucket = S3_BUCKET

In [ ]:
from src.db_building.corpus_building import (
    build_or_use_from_cache, DEFAULT_LOCATIONS,
)

In [6]:
modelid = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"

In [ ]:
df, all_splits = build_or_use_from_cache(
        model_id=modelid,
        embedding_model=modelid,
        filesystem=fs,
        s3_bucket=s3_bucket,
        location_dataset=DEFAULT_LOCATIONS,
        use_tokenizer_to_chunk = True,
    ) 

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = modelid
embedding_device = "cuda"

emb_model = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs={"device": embedding_device},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
    show_progress=False,
)

In [22]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="insee_data",
    embedding_function=emb_model,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
from langchain_core.documents import Document

ids = [i for i, _ in enumerate(all_splits)]

vector_store.add_documents(documents=all_splits, ids=ids)